In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Parameters
n_steps_in = 10  # Number of input steps
n_steps_out = 5  # Number of output steps
n_features = 2   # Number of input features
hidden_size = 50 # Number of LSTM units
num_epochs = 200 # Number of training epochs
output_features = 3  # Number of output features

# Generate dummy data
X = np.random.rand(1000, n_steps_in, n_features).astype(np.float32)
y = np.random.rand(1000, n_steps_out, output_features).astype(np.float32)

# Convert to PyTorch tensors
X = torch.from_numpy(X)
y = torch.from_numpy(y)

# Define the Encoder
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    
    def forward(self, x):
        out, (h, c) = self.lstm(x)
        return h, c

# Define the Decoder
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers=1):
        super(Decoder, self).__init__()
        self.lstm = nn.LSTM(output_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, h, c):
        out, (h, c) = self.lstm(x, (h, c))
        out = self.fc(out)
        return out, h, c

# Define the Seq2Seq model
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, n_steps_out, n_features, output_features):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.n_steps_out = n_steps_out
        self.n_features = n_features
        self.output_features = output_features
    
    def forward(self, x):
        h, c = self.encoder(x)
        decoder_input = torch.zeros((x.size(0), 1, self.output_features)).to(x.device)
        outputs = []
        for _ in range(self.n_steps_out):
            out, h, c = self.decoder(decoder_input, h, c)
            outputs.append(out)
            decoder_input = out
        outputs = torch.cat(outputs, dim=1)
        return outputs

# Initialize the model, loss function, and optimizer
encoder = Encoder(n_features, hidden_size)
decoder = Decoder(hidden_size, output_features)
model = Seq2Seq(encoder, decoder, n_steps_out, n_features, output_features)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Predict
model.eval()
X_new = torch.from_numpy(np.random.rand(1, n_steps_in, n_features).astype(np.float32))
y_pred = model(X_new)

print(y_pred)

Epoch [20/200], Loss: 0.1590
Epoch [40/200], Loss: 0.0863
Epoch [60/200], Loss: 0.0843
Epoch [80/200], Loss: 0.0837
Epoch [100/200], Loss: 0.0836
Epoch [120/200], Loss: 0.0836
Epoch [140/200], Loss: 0.0835
Epoch [160/200], Loss: 0.0835
Epoch [180/200], Loss: 0.0835
Epoch [200/200], Loss: 0.0835
tensor([[[0.5061, 0.5063, 0.5029],
         [0.5068, 0.4998, 0.5053],
         [0.5016, 0.4957, 0.5044],
         [0.4954, 0.4916, 0.5009],
         [0.4898, 0.4883, 0.4961]]], grad_fn=<CatBackward0>)


In [2]:
out,_ = encoder(X)

In [3]:
X.shape

torch.Size([1000, 10, 2])

In [4]:
out.shape

torch.Size([1, 1000, 50])